# Создаем телеграм Бот на основе модели.

Вызов модели, проверка

In [1]:
import pickle

with open('movie_recommender_model.pkl', 'rb') as f:
    model = pickle.load(f)

count_vectorizer = model['count_vectorizer']
cosine_sim = model['cosine_sim']
indices = model['indices']
movie_titles = model['movie_titles']

def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return movie_titles.iloc[movie_indices]

print(get_recommendations("The Dark Knight Rises"))

65               The Dark Knight
119                Batman Begins
4638    Amidst the Devil's Wings
1196                The Prestige
3073           Romeo Is Bleeding
3326              Black November
1503                      Takers
1986                      Faster
303                     Catwoman
747               Gangster Squad
Name: title, dtype: object


In [ ]:
import nest_asyncio
nest_asyncio.apply()
import pickle
import pandas as pd
import pytz
import asyncio
from telegram import Update
from telegram.ext import (
    Application,
    CommandHandler,
    MessageHandler,
    filters,
    ContextTypes,
)

# Загрузка модели
with open('movie_recommender_model.pkl', 'rb') as f:
    model = pickle.load(f)

count_vectorizer = model['count_vectorizer']
cosine_sim = model['cosine_sim']
indices = model['indices']
movie_titles = model['movie_titles']

def get_recommendations(title):
    try:
        idx = indices[title]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        movie_indices = [i[0] for i in sim_scores]
        return movie_titles.iloc[movie_indices]
    except KeyError:
        return None

# Команды бота
async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "🎬 Привет! Я бот для рекомендации фильмов.\n"
        "Напиши название фильма, и я предложу похожие."
    )

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "📌 Просто отправь мне название фильма, который тебе нравится,\n"
        "и я найду 10 похожих фильмов.\n\n"
        "Команды:\n"
        "/start - начать работу\n"
        "/help - справка"
    )

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_message = update.message.text
    recommendations = get_recommendations(user_message)
    
    if recommendations is not None:
        response = "🍿 Похожие фильмы:\n\n" + "\n".join(recommendations)
    else:
        response = "❌ Фильм не найден. Проверьте название или попробуйте другой."
    
    await update.message.reply_text(response)

async def error_handler(update: Update, context: ContextTypes.DEFAULT_TYPE):
    print(f"Ошибка: {context.error}")
    await update.message.reply_text("⚠️ Произошла ошибка. Попробуйте позже.")

async def main():
    print("🚀 Запуск бота...")
    
    application = Application.builder() \
        .token("7236772325:AAEP-d4gsgNlVFVBH46itR9GYn_UivPpIYw") \
        .arbitrary_callback_data(True) \
        .build()
    
    application.add_handler(CommandHandler("start", start_command))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    application.add_error_handler(error_handler)
    
    print("🔎 Бот запущен и слушает сообщения...")
    await application.run_polling()

if __name__ == '__main__':
    try:
        asyncio.run(main())
    except KeyboardInterrupt:
        print("Бот остановлен")

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
import logging
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, CallbackContext


# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)
logger = logging.getLogger(__name__)


async def start(update: Update, context: CallbackContext) -> None:
    """Sends a welcome message when the /start command is issued."""
    await update.message.reply_text("Привет! Я бот. Спроси меня, как меня зовут.")


async def echo(update: Update, context: CallbackContext) -> None:
    """Echos the user message."""
    if update.message.text.lower() == "как тебя зовут?":
        await update.message.reply_text("11111")
    else:
        await update.message.reply_text(update.message.text)  # Echo other messages


def main() -> None:
    """Start the bot."""
    # Replace 'YOUR_TELEGRAM_BOT_TOKEN' with your actual bot token
    application = ApplicationBuilder().token("YOUR_TELEGRAM_BOT_TOKEN").build()

    # Handlers
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, echo))

    # Start the bot
    application.run_polling()


if __name__ == "__main__":
    main()

TypeError: Only timezones from the pytz library are supported